In [41]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from itertools import combinations
from tqdm import tqdm  # 引入tqdm来显示进度条

# ==============================================================================
# 第一部分：辅助函数 (数据处理与相似度计算)
# ==============================================================================

def haversine_distance(p1, p2):
    """计算两个经纬度点之间的球面距离（单位：米）。"""
    R = 6371000  # 地球平均半径
    lon1, lat1 = p1
    lon2, lat2 = p2
    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)
    dlon, dlat = lon2_rad - lon1_rad, lat2_rad - lat1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def string_to_trajectory(track_string: str) -> np.ndarray:
    """将'lon,lat#lon,lat'格式的字符串转换为Numpy轨迹数组。"""
    if not isinstance(track_string, str) or not track_string.strip():
        return np.empty((0, 2))
    points = []
    point_strings = track_string.strip().split('#')
    for p_str in point_strings:
        coords = p_str.split(',')
        if len(coords) == 2:
            try:
                # 保持 [经度, 纬度] 格式
                lon, lat = float(coords[0]), float(coords[1])
                points.append([lon, lat])
            except (ValueError, TypeError):
                # 如果转换失败，跳过这个点
                pass
    return np.array(points)

def lcss_similarity(traj_a: np.ndarray, traj_b: np.ndarray, epsilon: float) -> float:
    """使用动态规划计算两条轨迹的LCSS相似度 (0-1之间)。"""
    m, n = len(traj_a), len(traj_b)
    if m == 0 or n == 0:
        return 0.0

    lcss_matrix = np.zeros((m + 1, n + 1))
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if haversine_distance(traj_a[i-1], traj_b[j-1]) <= epsilon:
                lcss_matrix[i, j] = 1 + lcss_matrix[i-1, j-1]
            else:
                lcss_matrix[i, j] = max(lcss_matrix[i-1, j], lcss_matrix[i, j-1])
    
    lcss_length = lcss_matrix[m, n]
    return lcss_length / ((m+n)/2)

# ==============================================================================
# 第二部分：构建时序图的主函数
# ==============================================================================

def build_temporal_graph(df: pd.DataFrame, epsilon: float, sigma: float, window_size: str = '1h') -> pd.DataFrame:
    """
    从轨迹数据构建时序图。

    参数:
        df (pd.DataFrame): 包含轨迹数据的DataFrame。
        epsilon (float): LCSS的距离阈值（米），用于判断点是否匹配。
        sigma (float): 轨迹相似度阈值（0-1），用于判断是否连接用户。
        window_size (str): Pandas时间窗口大小（例如 '1H', '30min', '1D'）。

    返回:
        pd.DataFrame: 包含图连接信息的DataFrame，列为 [user_u, user_v, layer]。
    """
    # --- 1. 数据预处理 ---
    print("步骤 1/4: 数据预处理...")
    # 复制数据，避免修改原始DataFrame
    processed_df = df.copy()
    # 转换时间列为datetime对象
    processed_df['start_time'] = pd.to_datetime(processed_df['start_time'])
    # 将track字符串解析为Numpy数组，方便后续计算
    processed_df['trajectory'] = processed_df['track'].apply(string_to_trajectory)
    # 设置时间为索引，方便分片
    processed_df.set_index('start_time', inplace=True)
    
    # --- 2. 按时间窗口分组 ---
    print(f"步骤 2/4: 按 '{window_size}' 的时间窗口对数据进行分组...")
    grouped_by_window = processed_df.groupby(pd.Grouper(freq=window_size))
    
    edge_list = []
    
    print("步骤 3/4: 在每个时间窗口内计算用户轨迹相似度...")
    # 使用tqdm来可视化处理进度
    for window_start_time, group_df in tqdm(grouped_by_window, desc="处理时间窗口"):
        if len(group_df) < 2:
            continue # 如果窗口内订单少于2，无法形成连接，跳过
        
        # 为了处理一个用户在窗口内有多次出行的情况，我们按用户ID分组
        # 并将同一用户的所有轨迹合并为一个长轨迹
        user_trajectories = {}
        for userid, user_trips in group_df.groupby('userid'):
            # 过滤掉空轨迹并合并
            valid_trajectories = [traj for traj in user_trips['trajectory'] if traj.size > 0]
            if valid_trajectories:
                user_trajectories[userid] = np.vstack(valid_trajectories)

        # 获取当前窗口内有有效轨迹的用户列表
        users_in_window = list(user_trajectories.keys())
        
        # 如果活跃用户少于2，也跳过
        if len(users_in_window) < 2:
            continue

        # --- 3. 计算窗口内用户对的相似度 ---
        # 使用itertools.combinations高效生成所有用户对
        for user_u, user_v in combinations(users_in_window, 2):
            traj_u = user_trajectories[user_u]
            traj_v = user_trajectories[user_v]
            
            similarity = lcss_similarity(traj_u, traj_v, epsilon=epsilon)
            
            # --- 4. 如果相似度达到阈值，添加边 ---
            if similarity >= sigma:
                # 确保user_u始终小于user_v，避免重复边 (u,v) 和 (v,u)
                u, v = sorted((user_u, user_v))
                edge_list.append({
                    'user_u': u,
                    'user_v': v,
                    'layer': window_start_time
                })

    print("步骤 4/4: 创建最终的DataFrame...")
    if not edge_list:
        print("警告：没有找到任何符合条件的相似用户对。返回一个空的DataFrame。")
        return pd.DataFrame(columns=['user_u', 'user_v', 'layer'])
        
    result_df = pd.DataFrame(edge_list)
    # 去除在同一个时间层内可能出现的重复边
    result_df.drop_duplicates(inplace=True)
    
    return result_df

# ==============================================================================
# 第三部分：示例用法
# ==============================================================================
if __name__ == '__main__':
    data = pd.read_csv("transport_data/mobike_shanghai_sample_updated.csv")
    # --- 设置参数 ---
    # LCSS距离阈值：如果两个GPS点相距200米以内，则认为它们匹配
    EPSILON_METERS = 1000 
    # 相似度阈值：如果两条轨迹的LCSS相似度得分高于等于0.8，则连接
    SIGMA_THRESHOLD = 0.5
    # 时间窗口
    WINDOW = '1h'

    # --- 运行主函数 ---
    print("\n开始构建时序图...")
    temporal_graph_df = build_temporal_graph(
        df=data,
        epsilon=EPSILON_METERS,
        sigma=SIGMA_THRESHOLD,
        window_size=WINDOW
    )

    # --- 显示并保存结果 ---
    print("\n构建完成！结果如下：")
    print(temporal_graph_df)

    if not temporal_graph_df.empty:
        output_filename = 'temporal_graph_edges.csv'
        temporal_graph_df.to_csv(output_filename, index=False)
        print(f"\n结果已保存到文件: {output_filename}")


开始构建时序图...
步骤 1/4: 数据预处理...
步骤 2/4: 按 '1h' 的时间窗口对数据进行分组...
步骤 3/4: 在每个时间窗口内计算用户轨迹相似度...


处理时间窗口: 100%|██████████| 744/744 [3:06:24<00:00, 15.03s/it]  


步骤 4/4: 创建最终的DataFrame...

构建完成！结果如下：
        user_u  user_v               layer
0         1183    5463 2016-08-01 00:00:00
1         4666    6667 2016-08-01 00:00:00
2           95    3356 2016-08-01 06:00:00
3          719    7733 2016-08-01 06:00:00
4         1440    4123 2016-08-01 06:00:00
...        ...     ...                 ...
107191   12929   13470 2016-08-31 23:00:00
107192   13416   14714 2016-08-31 23:00:00
107193   13459   14714 2016-08-31 23:00:00
107194   13470   14042 2016-08-31 23:00:00
107195   15764   16452 2016-08-31 23:00:00

[107196 rows x 3 columns]

结果已保存到文件: temporal_graph_edges.csv
